## Importing Libraries

In [13]:
# !pip install psycopg2
import time
import pandas as pd
import requests
import psycopg2
from datetime import datetime
import time

## Fetching Api

In [20]:
api_url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd", 
    "order": "market_cap_desc",
    "per_page": 250, 
}
total_pages=5

In [22]:
# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="crypto", user="postgres", password="root", host="localhost", port="5432"
)
cursor = conn.cursor()

In [24]:
for page in range(1, total_pages + 1):
    params["page"] = page  # Set the page number
    
    response = requests.get(api_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        print(f"Fetched {len(data)} records from page {page}")

        for coin in data:
            coin_id = coin['id']
            name = coin['name']
            symbol = coin['symbol']
            market_cap_rank = coin['market_cap_rank']

            cursor.execute("""
                INSERT INTO cryptocurrencies (coin_id, name, symbol, market_cap_rank)
                VALUES (%s, %s, %s, %s)
                ON CONFLICT (coin_id) DO NOTHING;
            """, (coin_id, name, symbol, market_cap_rank))

        conn.commit()
    else:
        print(f"Error fetching data from page {page}: {response.status_code}")
    
    time.sleep(2) 

# Close the connection
cursor.close()
conn.close()

print("Cryptocurrency data inserted successfully!")

Fetched 250 records from page 1
Fetched 250 records from page 2
Fetched 250 records from page 3
Fetched 250 records from page 4
Fetched 250 records from page 5
Error fetching data from page 6: 429
Error fetching data from page 7: 429
Error fetching data from page 8: 429
Error fetching data from page 9: 429
Error fetching data from page 10: 429
Cryptocurrency data inserted successfully!


In [46]:
# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="crypto", user="postgres", password="root", host="localhost", port="5432"
)
cursor = conn.cursor()
for page in range(1,total_pages+1):
    params['page']=page
    response = requests.get(api_url, params=params)
    if response.status_code==200:
        data = response.json()
        print(f"Fetched {len(data)} records from page {page}")

        for coin in data:
            coin_id = coin['id']
            circulating_supply = coin.get('circulating_supply') 
            total_supply = coin.get('total_supply')
            max_supply = coin.get('max_supply')
            ath = coin.get('ath')
            ath_date = coin.get('ath_date')
            atl = coin.get('atl')
            atl_date = coin.get('atl_date')
            # Check if coin exists in cryptocurrencies table
            cursor.execute("SELECT 1 FROM cryptocurrencies WHERE coin_id = %s", (coin_id,))
            if cursor.fetchone() is None:
                cursor.execute("""
        INSERT INTO cryptocurrencies (coin_id, name, symbol) 
        VALUES (%s, %s, %s)
        ON CONFLICT (coin_id) DO NOTHING;
    """, (coin_id, coin['name'], coin['symbol'],coin['market_cap_rank']))
                conn.commit()

            cursor.execute("""
        INSERT INTO coin_supply (
            coin_id, circulating_supply, total_supply, max_supply, ath, ath_date, atl, atl_date
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (coin_id) DO UPDATE SET
            circulating_supply = EXCLUDED.circulating_supply,
            total_supply = EXCLUDED.total_supply,
            max_supply = EXCLUDED.max_supply,
            ath = EXCLUDED.ath,
            ath_date = EXCLUDED.ath_date,
            atl = EXCLUDED.atl,
            atl_date = EXCLUDED.atl_date;
    """, (coin_id, circulating_supply, total_supply, max_supply, ath, ath_date, atl, atl_date))

            conn.commit()

    else:
        print(f"Error fetching data from page {page}: {response.status_code}")
    time.sleep(3)

cursor.close()
conn.close()

print("supply data inserted successfully!")

Fetched 250 records from page 1
Fetched 250 records from page 2
Fetched 250 records from page 3
Fetched 250 records from page 4
Fetched 250 records from page 5
Error fetching data from page 6: 429
Error fetching data from page 7: 429
Error fetching data from page 8: 429
Error fetching data from page 9: 429
Error fetching data from page 10: 429
supply data inserted successfully!


In [ ]:
# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="crypto", user="postgres", password="root", host="localhost", port="5432"
)
cursor = conn.cursor()
for page in range(1,total_pages+1):
    params['page']=page
    response = requests.get(api_url, params=params)
    if response.status_code==200:
        data = response.json()
        print(f"Fetched {len(data)} records from page {page}")

        for coin in data:
            coin_id = coin['id']
            current_price= coin['current_price']
            market_cap=coin['market_cap']
            total_volume=coin['total_volume']
            price_change_24h = coin.get('price_change_24h', None)
            price_change_percentage_24h = coin.get('price_change_percentage_24h', None)
            market_cap_change_24h = coin.get('market_cap_change_24h', None)
            market_cap_change_percentage_24h = coin.get('market_cap_change_percentage_24h', None)
            last_updated = coin['last_updated']
            cursor.execute("SELECT 1 FROM cryptocurrencies WHERE coin_id = %s", (coin_id,))
            if cursor.fetchone() is None:
                cursor.execute("""
            INSERT INTO cryptocurrencies (coin_id, name, symbol, market_cap_rank)
            VALUES (%s, %s, %s, %s)
            """, (coin_id, coin['name'], coin['symbol'], coin['market_cap_rank'])) 
                conn.commit()
            cursor.execute("""
    INSERT INTO coin_prices (
        coin_id, current_price, market_cap, total_volume, 
        price_change_24h, price_change_percentage_24h, 
        market_cap_change_24h, market_cap_change_percentage_24h, last_updated
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (coin_id) DO UPDATE SET
        current_price = EXCLUDED.current_price,
        market_cap = EXCLUDED.market_cap,
        total_volume = EXCLUDED.total_volume,
        price_change_24h = EXCLUDED.price_change_24h,
        price_change_percentage_24h = EXCLUDED.price_change_percentage_24h,
        market_cap_change_24h = EXCLUDED.market_cap_change_24h,
        market_cap_change_percentage_24h = EXCLUDED.market_cap_change_percentage_24h,
        last_updated = EXCLUDED.last_updated;
""", (coin_id, current_price, market_cap, total_volume, 
      price_change_24h, price_change_percentage_24h, 
      market_cap_change_24h, market_cap_change_percentage_24h, last_updated))
            conn.commit()
    else:
        print(f"Error fetching data from page {page}: {response.status_code}")
    time.sleep(3)

cursor.close()
conn.close()

print("price data inserted successfully!")

In [32]:
HISTORICAL_DATA_URL = "https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
DB_PARAMS = {
    "dbname": "crypto",
    "user": "postgres",
    "password": "root",
    "host": "localhost",
    "port": "5432",
}
CURRENCY = "usd"
DAYS = "30"
RATE_LIMIT = 30 
DELAY =4
conn = psycopg2.connect(**DB_PARAMS)
cur = conn.cursor()
def get_tracked_coins():
    cur.execute("SELECT coin_id FROM cryptocurrencies ORDER BY market_cap_rank ASC LIMIT 50;")
    coins = [row[0] for row in cur.fetchall()]
    return coins


In [34]:
def fetch_historical_data(coin_id, retries=5):
    url = HISTORICAL_DATA_URL.format(coin_id=coin_id)
    params = {"vs_currency": CURRENCY, "days": DAYS, "interval": "daily"}

    for attempt in range(retries):
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:  # Too many requests
                wait_time = (attempt + 1) * 10  # Exponential backoff (10s, 20s, 30s)
                print(f"Rate limit hit for {coin_id}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"Error fetching {coin_id}: {e}")
                return None
    return None

In [36]:
def store_historical_data(coin_id, data):
    if not data:
        return

    prices = data.get("prices", [])
    market_caps = data.get("market_caps", [])
    total_volumes = data.get("total_volumes", [])

    for i in range(len(prices)):
        timestamp = datetime.utcfromtimestamp(prices[i][0] / 1000)  # Convert ms to sec
        price = prices[i][1]
        market_cap = market_caps[i][1] if market_caps else None
        total_volume = total_volumes[i][1] if total_volumes else None

        cur.execute(
            """
            INSERT INTO historical_prices (coin_id, currency, price, market_cap, total_volume, timestamp)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (coin_id, timestamp) DO NOTHING;
            """,
            (coin_id, CURRENCY, price, market_cap, total_volume, timestamp),
        )

    conn.commit()

In [21]:
# Step 4: Run the Data Fetching Loop
def main():
    coin_list = get_tracked_coins()
    print(f"Fetching historical data for {len(coin_list)} coins...")

    batch_size = 30  
    long_wait = 90 
    short_wait = 5

    for index, coin_id in enumerate(coin_list):
        print(f"Fetching data for {coin_id} ({index + 1}/{len(coin_list)})...")
        data = fetch_historical_data(coin_id)
        store_historical_data(coin_id, data)

        # Respect rate limits
        if (index + 1) % batch_size == 0:
            print(f"Completed {batch_size} requests. Waiting for {long_wait} seconds...")
            time.sleep(long_wait)
        else:
            time.sleep(short_wait)  

    print("Data fetching completed!")

In [38]:
# Run the script
if __name__ == "__main__":
    main()

# Close DB connection
cur.close()
conn.close()

Fetching historical data for 50 coins...
Fetching data for bitcoin (1/50)...


C:\Users\Admin\AppData\Local\Temp\ipykernel_8624\2743609649.py:10: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  timestamp = datetime.utcfromtimestamp(prices[i][0] / 1000)  # Convert ms to sec


Fetching data for ethereum (2/50)...
Fetching data for ripple (3/50)...
Fetching data for tether (4/50)...
Fetching data for binancecoin (5/50)...
Fetching data for solana (6/50)...
Fetching data for usd-coin (7/50)...
Rate limit hit for usd-coin. Retrying in 10s...
Rate limit hit for usd-coin. Retrying in 20s...
Rate limit hit for usd-coin. Retrying in 30s...
Fetching data for dogecoin (8/50)...
Fetching data for cardano (9/50)...
Fetching data for staked-ether (10/50)...
Fetching data for tron (11/50)...
Fetching data for wrapped-bitcoin (12/50)...
Fetching data for chainlink (13/50)...
Rate limit hit for chainlink. Retrying in 10s...
Rate limit hit for chainlink. Retrying in 20s...
Rate limit hit for chainlink. Retrying in 30s...
Fetching data for wrapped-steth (14/50)...
Fetching data for stellar (15/50)...
Fetching data for avalanche-2 (16/50)...
Fetching data for sui (17/50)...
Fetching data for litecoin (18/50)...
Rate limit hit for litecoin. Retrying in 10s...
Rate limit hit fo